<a href="https://colab.research.google.com/github/james-trayford/AudibleUniverseWorkbooks/blob/group3/STRAUSSdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook prepared by **Dr James Trayford** - for queries please email [`james.trayford@port.ac.uk`](mailto:james.trayford@port.ac.uk)

To use this notebook (if you haven't already) you can first save a copy to your local drive by clicking `File > Save a Copy in Drive` and run on that copy. `Edit > Clear all outputs` on that copy should also ensure yopu have a clean version
 to start from.

## **0. Introduction**




We will be using the [STRAUSS code](https://github.com/james-trayford/strauss) for this activity

<img src="https://github.com/james-trayford/strauss/blob/main/misc/strauss_logo.png?raw=true">

For reference, you can read an overview of the code (as well as detailed documentation) [at this link](https://strauss.readthedocs.io/en/latest/)


`strauss` is an open source, object-oriented python library intended to be a flexible toolkit and engine for sonification, allowing detailed low-level control over the sonification process. Simultaneously, casual users can quickly hear their data, adapting a library of python notebook templates for a range of applications. 

By analogy to visualisation, the intention is to provide something akin to a plotting library. A library allows users to make a variety of simple plots easily, but also the option to control all aspects of plots and adapt them to the intricacies of their data, for optimal presentation. 

By adopting a general approach, `strauss` is intended to sonify any form of data for users with differing expertise. `strauss` is work in progress, and benefits form user feedback - filling in this feedback will be very useful in making the code better!


### **This notebook:** 
This notebook will demonstrate some of the ways in which `strauss` can be applied to sonify spectra. Alternative options may be demonstrated with commented out code ( i.e. lines of actual code preceded by `#`) - feel free to try these! Generally the goal of this notebook is to give some open examples to explore the code and experiment, so please do so! 

### **STRAUSS video**

You can also run the below cell to see a 12 minute introduction video

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('yhSNM8ztSEk', width=800, height=600) 

## **1. Setup:**



First, let's install `strauss`! Just run the code cell below.

*We will use the `spectraliser` development branch for this notebook. Install can take a while - but you should only need to run it once!*


In [ ]:
 !pip --quiet install git+https://github.com/james-trayford/strauss.git@spectraliser -U

and also make a local copy of the repository

In [ ]:
 !git clone https://github.com/james-trayford/strauss.git

Make plots appear in-line by default

In [ ]:
%matplotlib inline

Import the modules we need...

In [ ]:
# strauss imports
from strauss.sonification import Sonification
from strauss.sources import Events, Objects
from strauss import channels
from strauss.score import Score
from strauss.generator import Sampler, Synthesizer, Spectralizer
from strauss import sources as Sources 
import strauss

# other useful modules
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.signal import savgol_filter
import urllib.request
import os
import zipfile
import glob
import yaml

# modules to display in-notebook
import IPython.display as ipd
from IPython.core.display import display, Markdown, Latex, Image

# set figures to be a decent size by default
import matplotlib
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)
matplotlib.rc('figure', **{'figsize':[14.0, 7.0]})

## **2. Getting the Data**

⚠  ***The cells in this section are to organise the data we need for the session, don't worry too much about the details of the code here!*** ⚠ 

First, let's download the data for this session to somewhere we have `Colab` access

In [ ]:
outdir = "./AU2_Group3"

path = os.path.realpath(outdir)
if not glob.glob(outdir): 
  os.mkdir(path)  
    
fname = "Group3_input_data.zip"
url = "https://drive.google.com/uc?export=download&id=1rLKWbajD6PV9qbMuWDrq0SaDnDCQSaSc"

print(f"Downloading files...")
with urllib.request.urlopen(url) as response, open(f"{path}/{fname}", 'wb') as out_file:
  data = response.read() # a `bytes` object
  out_file.write(data)

print(f"Unzipping files to {outdir}/Data ...")
with zipfile.ZipFile(f"{outdir}/{fname}", 'r') as zip_ref:
    zip_ref.extractall(f"{outdir}")

print(f"Clearing up...")
os.remove(f"{path}/{fname}")

print("Done.")

Next, let's ***visually display these spectra*** and make a couple of ***data structures*** for easy 
access ...

In [ ]:
types = ["Type 1", "Type 1.5", "Type 2"]

spectra = {}
spectra_plots = {}

for t in types:
  display(Markdown(f"### **{t} spectra:**"))
  spectra_plots[t] = {}
  spectra[t] = {}
  for img in glob.glob(f"./AU2_Group3/Data/{t.replace(' ', '')}/*.png"):
    display(Image(filename = img))
  for csv in glob.glob(f"./AU2_Group3/Data/{t.replace(' ', '')}/*.csv"):
    spectrum = np.genfromtxt(csv, delimiter=',')[1:]
    label = '-'.join(csv.split('/')[-1].split('.')[0].split('-')[1:])
    spectra[t][label] = spectrum
    spectra_plots[t][label] = '.'.join(csv[:-3].split('.')[:-1]) + ".png"


We now have access to all the spectra in the `data` object - let's see what's available for ***Type 1*** first...

In [ ]:
# show available types:
print("Spectral types: \n", list(spectra.keys()), '\n')

# show available 'Type 1' examples:
print("Type 1 spectra: \n", list(spectra["Type 1"].keys()))

Finally, let's generate a plot of one of the ***Type 1*** spectrum `'51788-0386-086'` ourselves.

In [ ]:
# we stored the spectra as numpy arrays, where axis 0 is wavelength...
wavelength = spectra['Type 1']['51788-0386-086'][:,0]

# ... and axis 1 is flux density...
flux = spectra['Type 1']['51788-0386-086'][:,1]

plt.plot(wavelength, flux)
plt.ylabel('Flux Density')
plt.xlabel('Wavelength [nm]')
plt.show()

## **3. One-dimensional time series sonification**

Here we will sonify spectra as a ***one-dimensional time series*** , where some ***sound property*** is is varied with ***time*** in the ***sonification***, in the same way that **flux density**, varies with ***wavelength*** in a ***spectrum*** (early in the sonification is shorter wavelengths, later is longer wavelengths). This approach in general is covered in more detail in the [other activity workbook](https://github.com/james-trayford/AudibleUniverseWorkbooks/blob/group4/STRAUSSdemo.ipynb)!

### 3.1 Trying the `Events` source function

In `strauss` we could treat each ***flux density*** data point in the spectra as separate audio `Events`, with an occurence `time` mapped from their ***wavelength***. 

However, articulating each data point as a separate ***note*** for ***many thousands*** of data points can require long and drawn-out sonifications. 

Here we demonstrate this approach with just a portion of the data points (staying within `Colab`'s RAM limitations for unpaid users 🤫). We use the `Synthesizer` object with the `pitch_mapper` preset by default - this has a default pitch range of ***two octaves*** (a factor of 4 in frequency) and we pick an `E3` note (165 Hz) as the base (lowest) frequency.

We will hear the **flux** of each point as `pitch`, with their wavelength mapped to the occurence `time` (moving from low to high wavelength). we can hear the general descent and some emission peaks, as well as the fact that the wavelength spacing gets **wider** as we go on. 


In [ ]:
display(Markdown(f"### Sonifying in 1D using '`Events`' object:"))

# show spectrum again, for reference
plt.scatter(wavelength[1000:2000], flux[1000:2000], s=4)
plt.ylabel('Flux Density')
plt.xlabel('Wavelength [nm]')
plt.show()

%matplotlib notebook
# specify the base notes used. In this example we use a single E3 note and 
# freely vary the pitch via the 'pitch_shift' parameter 
notes = [["E3"]]

# we could also just specify a particular frequency...
#notes = [[150.]]

score =  Score(notes, 80)


data = {'pitch':np.ones(flux.size)[1000:2000],
        'time': wavelength[1000:2000],
        'pitch_shift':flux[1000:2000]}

# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# set up synth (this generates the sound using mathematcial waveforms)
generator = Synthesizer()
generator.load_preset('pitch_mapper')

# or maybe the sampler instead by uncommenting this block (this uses recorded audio clips)
# generator = Sampler(sampfiles="./strauss/data/samples/mallets")
# generator.modify_preset({'phi': 0,'theta':0,})

generator.modify_preset({'note_length':0.1,
                         'volume_envelope': {'use':'on',
                                             'D':0.1,
                                             'S':0.,
                                             'A':0.001}})

# set 0 to 100 percentile limits so the full pitch range is used...
# setting 0 to 101 for pitch means the sonification is 1% longer than 
lims = {'time': ('0','101'),
        'pitch_shift': ('0','100')}

# set up source
sources = Events(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

The articulation of each note lets you hear each data point separately, but over many noisy data points can lead to a confusing result - it's hard to keep track of the fluxes and our pitch memory is challenged.

Instead we can try ***smoothly evolving*** parameters, designating the spectra as an ***evolving `Object`*** source class. We demonstrate this in the following subsection `3.1`.

### 3.2 Instead Trying the `Objects` Source Type

Let's first try the evolving `Object` approach without modifying the raw data in any way. In analogy to visual display, the `Event` representation is like plotting the spectrum as a scatter plot, while an `Object` representation is like plotting the spectrum as a continuous line.

 We set up some parameters for `strauss`, e.g. choosing a sonification length of `40` seconds. A longer sonification might let you hear more detail, but will take longer to listen to (and for `Colab` to load!)

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# length of the sonification in s
length = 40.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.load_preset('pitch_mapper')
generator.preset_details('pitch_mapper')

Let's pick a spectrum to sonify (as in Section 2). We will again default to ***Type 1*** spectrum `'51788-0386-086'`.

Try changing this if you want to explore different spectra!

In [ ]:
stype = "Type 1"
slabel = "51788-0386-086"

# we stored the spectra as numpy arrays, where axis 0 is wavelength...
wavelength = spectra[stype][slabel][:,0]

# ... and axis 1 is flux density.
flux = spectra[stype][slabel][:,1]

# Finally, can use this to remind ourselves what the spectrum looks like...
display(Image(spectra_plots[stype][slabel]))

let's hear the 1D time-series data sonification, mapping flux density to pitch for the spectrum.

*NB: If you were wondering why `strauss` refers to the varying pitch mapping as `pitch_shift` and not just `pitch`, this is because all sources in `strauss` also need a base `pitch` which is chosen from the `'notes'` structure (here always `'A2'`) by the `Score`. This is because `strauss` can play many sources at the same time! Again, you can read more about this [in the docs](https://strauss.readthedocs.io/en/latest/).*

In [ ]:
display(Markdown(f"### Sonifying in 1D using '`pitch_shift`':"))

# show spectrum again, for reference
display(Image(spectra_plots[stype][slabel]))

%matplotlib notebook
notes = [["A2"]]
score =  Score(notes, length)

data = {'pitch':1.,
        'time_evo':wavelength,
        'pitch_shift':flux}

# set 0 to 100 percentile limits so the full pitch and time range is used...
lims = {'time_evo': ('0','100'),
        'pitch_shift': ('0','100')}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

How about mapping a low-pass filter to flux density? 

*using a 'tonal' carrier, uncomment line for a 'textural' (or white noise) carrier*

In [ ]:
display(Markdown(f"### Sonifying in 1D using low-pass '`cutoff`' frequency:"))

# show spectrum again, for reference
display(Image(spectra_plots[stype][slabel]))

%matplotlib notebook
generator = Synthesizer()
generator.modify_preset({'filter':'on'})

# uncomment these lines to try a 'textural' sonification using white noise!
# generator.load_preset('windy')
# generator.preset_details('windy')

# we use a (power!) 'chord' here to create more harmonic richness...
notes = [["A2", "E3", 'A3', 'E4']]
score =  Score(notes, length)

data = {'pitch':[0,1,2,3],
        'time_evo':[wavelength]*4,
        'cutoff':[flux]*4}

lims = {'time_evo': ('0','100'),
        'cutoff': ('0','100')}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
plims = {'cutoff': (0.25,0.9)}
sources.apply_mapping_functions(map_lims=lims, param_lims=plims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()

# change back in case cells are run out of order...
generator.load_preset('pitch_mapper')
%matplotlib inline

In fact, there are many expressive properties of sound we could use to represent the data in a similar way. In `strauss` these are referred to as `mappable` properties. 

A subset of these can be used as an evolving property with the `Object` source class. These are referred to as `evolvable` properties. 

Lets show what's available:

In [ ]:
display(Markdown(f"### ***'Mappable'*** properties:"))
for m in Sources.mappable:
  display(Markdown(f' * `{m}` '))

display(Markdown(f"### ***'Evolvable'*** properties:"))
for m in Sources.evolvable:
  display(Markdown(f' * `{m}` '))

We can play around with some of these `evolvable` properties here. Are all of these effective for this data? Could they be effective for other types of data representations?

The `idx` variable below controls which evolvable property is selected from the `some_mappings` list. `idx` can be changed to a number from 0 to 5 inclusive to *'index'* a certain sound parameter.

In [ ]:
# A list of some 'evolvable' mappings
some_mappings = ["volume", 
                 "phi",
                 "volume_lfo/amount", 
                 "volume_lfo/freq_shift",
                 "pitch_lfo/amount", 
                 "pitch_lfo/freq_shift"]

# change this (between 0 and 5) to select a different property to map...
idx = 0

# use a stereo system to allow 'phi' mapping (low pan left and high pan right)
system = "stereo"

display(Markdown(f"### Sonifying in 1D using `evolvable` property - ***`{some_mappings[idx]}`***:"))

# show spectrum again, for reference
display(Image(spectra_plots[stype][slabel]))

%matplotlib notebook
generator = Synthesizer()
generator.modify_preset({'filter':'on',
                         "pitch_hi":-1, "pitch_lo": 1,
                         "pitch_lfo": {"use": "on", 
                                       "amount":1*("pitch_lfo/freq_shift" == some_mappings[idx]), 
                                       "freq":3*5**("pitch_lfo/freq_shift" != some_mappings[idx]), 
                                       "phase":0.25},
                         "volume_lfo": {"use": "on", 
                                        "amount":1*("volume_lfo/freq_shift" == some_mappings[idx]), 
                                        "freq":3*5**("volume_lfo/freq_shift" != some_mappings[idx]), 
                                        "phase":0}
                        })

# try a different chord (stacking fifths)...
notes = [["A2","E3","B4","F#4"]]

# chords and can also be specified via chord names and base octave....
# notes = "Em6_3"


score =  Score(notes, length)

data = {'pitch':[0,1,2,3],
        'time_evo':[wavelength]*4,
        'cutoff':[0.9]*4,
        'theta':[0.5]*4,
        some_mappings[idx]:[flux/flux.max()]*4}

lims = {'time_evo': ('0','100'),
        "volume": ('0','100'), 
        "phi": (-0.5,1.5),
        "volume_lfo/amount": ('0','100'), 
        "volume_lfo/freq_shift": ('0','100'),
        "pitch_lfo/amount": ('0','100'), 
        "pitch_lfo/freq_shift": ('0','100')}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
plims = {'cutoff': (0.25,0.9)}
sources.apply_mapping_functions(map_lims=lims, param_lims=plims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

# change back in case cells are run out of order...
generator.load_preset('pitch_mapper')
system = 'mono'


### 3.X A Side Note About Presets

You can see all the parameters that make up the default `Synthesizer` preset in a pop-up window:

In [ ]:
%pycat /usr/local/lib/python3.7/dist-packages/strauss/presets/synth/default.yml

... as well as the other presets we might want to use

In [ ]:
ls -1 /usr/local/lib/python3.7/dist-packages/strauss/presets/synth/*.yml

We can modify these presets at runtime (as we do in various examples throughout this session), or even write our own presets in `.yml` format.

The `generator.preset()` function also accepts a filepath to a custom presets, where any changed preset parameters replace those in the `default` preset. 

## **4. "Spectralizer" sonification**

Now, we will try converting the ***light spectrum*** into a ***sound spectrum*** directly, i.e. the sound has the same frequency features as the observed spectrum, albeit at ***audible frequencies*** 

Set up some STRAUSS stuff...

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"
length = 10

# set uo score object for sonification
score =  Score(["C3"], length)

sonify the spectrum...

In [ ]:
# show spectrum again, for reference
display(Image(spectra_plots[stype][slabel]))

# set up spectralizer generator
generator = Spectralizer()

# set up spectrum and choose some envelope parameters for fade-in and fade-out
data = {'spectrum':[flux[::-1]], 'pitch':[1],
        'volume_envelope/D':[0.9], 
        'volume_envelope/S':[0.], 
        'volume_envelope/A':[0.05]}

# again, use maximal range for the mapped parameters
lims = {'spectrum': ('0','100')}

# set up source
sources = Events(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions(map_lims=lims)

# render and play sonification!
soni = Sonification(score, sources, generator, system)
soni.render()
soni.save('spectralize1.wav')
ipd.Audio('spectralize1.wav')

We can try the same thing, but what if we vary the ***sound frequency range*** the spectrum is mapped to? 

This can be done by changing the `Spectralizer` generator preset: 

In [ ]:
# we can modify the frequency range (in Hz) like this: 
generator.modify_preset({'min_freq':500, 'max_freq':2000})

# We could try any range we like (though Colab might crash from lack of RAM if we try extreme options...)
#generator.modify_preset({'min_freq':20, 'max_freq':22000})
#generator.modify_preset({'min_freq':250, 'max_freq':4000})

# Particularly, if we want to preserve harmonic relations, need to set the minimum and maximum frequency 
# such that the sonified spectrum spans the same range in log frequency as the original light spectrum
# can do this in the loop to ensure this is true for each spectrum 

#generator.modify_preset({'min_freq':100, 'max_freq':100*min(wavelength)/max(wavelength)})  

# render and play sonification!
soni = Sonification(score, sources, generator, system)
soni.render()
soni.save('spectralize2.wav')
display(ipd.Audio('spectralize2.wav'))

# reset preset to default now we've made the sonification...
generator.load_preset()

Are certain ranges better for hearing features? Is using the full range of human hearing (about 20 Hz to 22kHz) optimal?

Can you set up a way to try different ranges systematically?

## **5. Iterating through examples & pre-processing data**

🚧 ***under construction*** 🚧

### **5.1 Cycling through the spectra**


Let's iterate through all these spectra we've got:

In [ ]:
for t in spectra.keys():
    display(Markdown(f"### **{t} spectra:**"))
    for s in spectra[t].keys():
      display(Markdown(f"* ID <mark>`{s}`</mark> (available as <mark>`spectra['{t}']['{s}']`</mark>)"))

For example, Let's spectralise each of these `'Type 1'` spectra 

In [ ]:
# can set up generator, limits and most of the data mapping dictionary 
# outside of the loop  
generator = Spectralizer()

data = {'pitch':[1],
        'volume_envelope/D':[0.9], 
        'volume_envelope/S':[0.], 
        'volume_envelope/A':[0.05]}

lims = {'spectrum': ('0','100')}

# now loop through spectra...
for t in spectra.keys():

  # only do stuff for the 'Type 1's... can change this or comment out!
  if t == "Type 1":
    display(Markdown(f"### **{t} spectra:**"))
    for s in spectra[t].keys():
      wavelength = spectra[t][s][:,0]
    
      # if we want to preserve harmonic relations, need to set the minimum and maximum frequency 
      # such that the sonified spectrum spans the same range in log frequency as the original light spectrum
      # can do this in the loop to ensure this is true for each spectrum 
      
      #generator.modify_preset({'min_freq':100, 'max_freq':100*min(wavelength)/max(wavelength)})  
        
      flux = spectra[t][s][:,1]
      
      # show spectrum again, for reference
      display(Image(spectra_plots[t][s], width=500,height=400))
      display(Markdown(f"* ID <mark>`{s}`</mark>"))

      # set the spectrum to use...
      data['spectrum'] = [flux[::-1]]


      # set up source
      sources = Events(data.keys())
      sources.fromdict(data)
      sources.apply_mapping_functions(map_lims=lims)

      # render and play sonification!
      soni = Sonification(score, sources, generator, system)
      soni.render()
      soni.save(f"spectralize_{t.replace(' ', '')}_{s}.wav")
      display(ipd.Audio(f"spectralize_{t.replace(' ', '')}_{s}.wav"))

The noisy component you hear in some of these examples is associated with ***the continuum*** (the shape of the spectrum ignoring the spiky features). Having a non-zero flux density over a broad range of wavelengths leads to a ***'noisy'*** sound as tone combine over a broad range in frequency. We will address this in the next subsection `5.2`...

### **5.2 Pre-processing the data: Subtracting a continuum**

One way we can process this data is by ***fitting a continuum*** and ***subtracting it away***, leaving only those tasty ***spectral lines***. We'll also fit to the **logarithm** of the flux, which will stop the spiky features influencing the fit so much. Here's a simple function to do that fitting 

*Rough fits for now, but demonstrates the effect! - can you do better?*

In [ ]:
# fit polynomial continuum to x-y dat and choosing a polynomial 'order' 
# (i.e. for order=3 we find A,B,C,D for the Ax^3 + Bx^2 + Cx + D that best fits)
# we mask out sharp spikes and fit to the inverted spectrum to try and represent
# the base continuum level. 
def log_poly(x,y,order=10):
    # set a minimum flux to avoid breaking log function
    yclip = np.clip(y, 10,np.inf)
    lyclip = np.log10(yclip)
    bdx = yclip.astype(bool)
    lysmooth = savgol_filter(np.log10(yclip),501, 3)
    bdx = lyclip < lysmooth
    p = np.polyfit(x[bdx],1/yclip[bdx],order).reshape(1, order+1)
    exps = np.arange(order+1)[::-1].reshape(1,order+1)
    ypol = 1/((pow(x.reshape(x.size,1), exps) * p).sum(1))
    return ypol

So let's do the same thing again, but now subtract the continuum away and clip values below the continuum to just get ***emission lines***.

We'll also make some plots to illustrate this:

In [ ]:
# can set up generator, limits and most of the data mapping dictionary 
# outside of the loop  
generator = Spectralizer()
#generator.modify_preset({'min_freq':250, 'max_freq':1000})

data = {'pitch':[1],
        'volume_envelope/D':[0.9], 
        'volume_envelope/S':[0.], 
        'volume_envelope/A':[0.05]}

lims = {'spectrum': ('0','100')}

# now loop through spectra...
for t in spectra.keys():

  # only do stuff for the 'Type 1's... can change this or comment out!
  #if t == "Type 1":
    display(Markdown(f"### **{t} spectra:**"))
    for s in spectra[t].keys():
      wavelength = spectra[t][s][:,0]
    
      # if we want to preserve harmonic relations, need to set the minimum and maximum frequency 
      # such that the sonified spectrum spans the same range in log frequency as the original light spectrum
      # can do this in the loop to ensure this is true for each spectrum 
        
      #generator.modify_preset({'min_freq':100, 'max_freq':100*min(wavelength)/max(wavelength)})
    
      flux = spectra[t][s][:,1]
      continuum = log_poly(wavelength, flux)

      # subtract continuum and take only positive fluxes...
      flux_csub = np.clip(flux - continuum, 0, np.inf)

      # set the spectrum to use...
      data['spectrum'] = [flux_csub[::-1]]

      # what would the continuum sound like on its own?
      #data['spectrum'] = [continuum[::-1]]

      # what about flipping the spectrum so we hear what's below the continuum?
      #data['spectrum'] = [np.clip(continuum - flux, 0, np.inf)[::-1]]

      # Plotting bit ========================================
      fig = plt.figure(figsize=(15,6))
      ax1 = fig.add_subplot(121)
      ax1.plot(wavelength, flux, label='Raw flux')
      ax1.plot(wavelength, continuum, lw=5, alpha=0.5, label='Continuum fit')
      plt.xlabel("Wavelength [nm]")
      plt.ylabel("Flux Density")   
      plt.legend(frameon=0)
      ax2 = fig.add_subplot(122)
      ax2.plot(np.linspace(generator.preset['min_freq'],
                           generator.preset['max_freq'],
                           flux.size), data['spectrum'][0], c='k', label='Sonified spectrum')  
      plt.xlabel("Frequency [Hz]") 
      plt.ylabel("Flux Density (Continuum Subtracted)")
      plt.ylim(-10,flux.max())   
      plt.legend(frameon=0)
      plt.show()
      display(Markdown(f"* ID <mark>`{s}`</mark>"))
      # =====================================================

      # set up source
      sources = Events(data.keys())
      sources.fromdict(data)
      sources.apply_mapping_functions(map_lims=lims)

      # render and play sonification!
      soni = Sonification(score, sources, generator, system)
      soni.render()
      soni.save(f"spectralize_{t.replace(' ', '')}_{s}.wav")
      display(ipd.Audio(f"spectralize_{t.replace(' ', '')}_{s}.wav"))

## **6. Sandbox & Miscellaneous Functions** 

🚧 ***under construction*** 🚧